In [8]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._

import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._


In [9]:
val inputStreamPath = "./../data/events-stream"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath: String = ./../data/events-stream
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(tweet,StringType,true))
inputDF: org.apache.spark.sql.DataFrame = [tweet: string]


In [10]:
val trainedModel = PipelineModel.load("./models/spark-ml-model-rf")
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

trainedModel: org.apache.spark.ml.PipelineModel = pipeline_e4bd24ccb25b
getProbability: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


In [11]:
val predictionDf = trainedModel.transform(inputDF)

predictionDf.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF
        .withColumn("probability_hw_result", getProbability($"probability"))
    .show()
}.start()

predictionDf: org.apache.spark.sql.DataFrame = [tweet: string, words: array<string> ... 6 more fields]
res2: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@44a9b628


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+---------------------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_hw_result|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+---------------------+
|@JordynsMom idk. ...|[@jordynsmom, idk...|(1000,[7,48,78,29...|(1000,[7,48,78,29...|[4.98574550380466...|[0.49857455038046...|       1.0|             1|   0.5014254496195333|
|@amykate Hmm, mig...|[@amykate, hmm,, ...|(1000,[22,38,82,8...|(1000,[22,38,82,8...|[4.88381070163822...|[0.48838107016382...|       1.0|             1|    0.511618929836178|
|@joelgoodman yike...|[@joelgoodman, yi...|(1000,[100,170,26...|(1000,[100,170,26...|[4.75950360372200...|[0.47595036037


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+---------------------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_hw_result|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+---------------------+
|  @mikead  Clean Me!|[@mikead, , clean...|(1000,[340,372,87...|(1000,[340,372,87...|[4.89510838998072...|[0.48951083899807...|       1.0|             1|    0.510489161001927|
|i havent felt thi...|[i, havent, felt,...|(1000,[170,329,37...|(1000,[170,329,37...|[4.89510838998072...|[0.48951083899807...|       1.0|             1|    0.510489161001927|
|@Garf87 ROFL i wa...|[@garf87, rofl, i...|(1000,[121,234,29...|(1000,[121,234,29...|[5.09110953988884...|[0.5091109539


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+---------------------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_hw_result|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+---------------------+
|@hannah106 omg ki...|[@hannah106, omg,...|(1000,[48,77,157,...|(1000,[48,77,157,...|[4.89510838998072...|[0.48951083899807...|       1.0|             1|    0.510489161001927|
|@marleisse I wish...|[@marleisse, i, w...|(1000,[213,329,49...|(1000,[213,329,49...|[4.89510838998072...|[0.48951083899807...|       1.0|             1|    0.510489161001927|
|Joe Nieuwendyk le...|[joe, nieuwendyk,...|(1000,[0,39,48,77...|(1000,[0,39,48,77...|[5.00742407698274...|[0.5007424076


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+---------------------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_hw_result|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+---------------------+
|@ImJusInCredible ...|[@imjusincredible...|(1000,[48,76,80,8...|(1000,[48,76,80,8...|[5.04087058978040...|[0.50408705897804...|       0.0|             0|   0.4959129410219596|
|don't think that ...|[don't, think, th...|(1000,[159,426,56...|(1000,[159,426,56...|[4.89510838998072...|[0.48951083899807...|       1.0|             1|    0.510489161001927|
|is a bit hungry a...|[is, a, bit, hung...|(1000,[14,62,86,1...|(1000,[14,62,86,1...|[5.06171353774002...|[0.5061713537